In [24]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

# Read and merge Census

In [25]:
data_path = "../data_subsidy/"
acs1 = pd.read_csv(data_path + 'ACSDP5Y2016.DP03_data_with_overlays_2022-03-17T201553.csv', na_values=['(X)']) #, encoding='cp1252'
acs2 = pd.read_csv(data_path + 'ACSDP5Y2016.DP05_data_with_overlays_2021-12-09T232536.csv', na_values=['(X)']) 
acs3 = pd.read_csv(data_path + 'ACSST5Y2016.S1901_data_with_overlays_2022-03-17T201851.csv', na_values=['(X)']) 
acs4 = pd.read_csv(data_path + 'ACSST5Y2016.S1701_data_with_overlays_2021-12-12T065723.csv', na_values=['(X)'])

In [26]:
def remove_cols(df):
    df.iloc[0]['GEO_ID'] = 'county_name'
    keywords =["Estimate", "county_name"]
    clean_df = pd.DataFrame(df.iloc[0].str.contains("|".join(keywords)))
    clean_df = clean_df.loc[clean_df[0]==True]
    df.drop(df.columns.difference(clean_df.index), 1, inplace=True)
    df['County'] = df['GEO_ID'].str[-5:]
    df['County'] = df['County'].astype(str)  
    df['County'] = df['County']
    df.drop(columns=['GEO_ID'], inplace=True)
    df.fillna(0, inplace = True)
    return df
dfs = [acs1, acs2, acs3, acs4]
for i in dfs:
    remove_cols(i)

In [27]:
acs1 = acs1.loc[1:]
acs2 = acs2.loc[1:]
acs3 = acs3.loc[1:]
acs4 = acs4.loc[1:]

In [28]:
from functools import partial, reduce
dfs = [acs1, acs2, acs3, acs4]
merge = partial(pd.merge, on=['County'], how='outer')
all_acs = reduce(merge, dfs)
all_acs

,DP03_0001E,DP03_0002E,DP03_0003E,DP03_0004E,DP03_0005E,DP03_0006E,DP03_0007E,DP03_0008E,DP03_0009E,DP03_0010E,...,S1701_C03_052E,S1701_C03_053E,S1701_C03_054E,S1701_C03_055E,S1701_C03_056E,S1701_C03_057E,S1701_C03_058E,S1701_C03_059E,S1701_C03_060E,S1701_C03_061E
0,42712,26008,25699,24262,1437,309,16704,25699,0,22074,...,20.2,20.2,36.7,23.8,19.3,8.7,0,2.1,42.8,40.2
1,160301,93872,93640,87753,5887,232,66429,93640,0,82813,...,31.9,19.7,16.7,28.4,9.8,17.2,0,4.8,37.6,35.5
2,21476,10316,10316,8993,1323,0,11160,10316,0,9938,...,25.2,26.9,40.4,35.6,40.5,27.0,0,1.9,45.7,50.5
3,18496,9002,8997,8354,643,5,9494,8997,0,8514,...,43.5,21.1,35.3,36.2,24.4,19.2,0,7.8,28.4,46.6
4,46007,22969,22960,21593,1367,9,23038,22960,0,23370,...,35.8,30.7,28.7,32.8,23.9,20.1,0,0.6,34.1,44.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,45100,17037,17037,14092,2945,0,28063,17037,0,23837,...,33.8,43.1,55.4,72.4,75.7,63.9,0,2.8,70.8,78.6
3216,7229,3342,3342,2932,410,0,3887,3342,0,3622,...,70.7,11.3,30.0,47.0,34.8,66.8,0,4.2,63.3,55.0
3217,19212,9278,9256,6960,2296,22,9934,9256,0,9970,...,65.1,58.1,57.4,72.0,70.2,79.3,0,8.8,81.9,81.6
3218,28916,10802,10778,8171,2607,24,18114,10778,0,15074,...,62.2,56.6,68.1,60.7,70.7,75.5,0,7.0,76.1,79.4


# Preprocess CSR

In [29]:
csr = pd.read_csv(data_path + 'csrzipcounty2016.csv', na_values=['.'])
csr.columns = csr.iloc[2]
csr = csr.iloc[4:-1]
csr.reset_index(drop=True)
csr

2,State,FIPS County Code,County Name,Total Number of Consumers,Number of Consumers with CSR (AV of 73%/87%/94%),Number of Consumers with CSR AV of 73%,Number of Consumers with CSR AV of 87%,Number of Consumers with CSR AV of 94%,Average Monthly Advanced CSR Payment for Consumers with 73%,Average Monthly Advanced CSR Payment for Consumers with 87%,Average Monthly Advanced CSR Payment for Consumers with 94%
4,AK,02013,ALEUTIANS EAST,38,16,NaN,NaN,NaN,NaN,NaN,NaN
5,AK,02016,ALEUTIANS WEST,38,16,NaN,NaN,NaN,NaN,NaN,NaN
6,AK,02020,ANCHORAGE,"9,484","3,917",787,"1,640","1,490",$26,$204,$290
7,AK,02050,BETHEL,73,18,NaN,11,NaN,NaN,$234,NaN
8,AK,02060,BRISTOL BAY BOROUGH,34,12,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2600,WY,56037,SWEETWATER,"1,168",665,109,222,334,$16,$119,$173
2601,WY,56039,TETON,"2,812","1,411",428,484,499,$15,$115,$165
2602,WY,56041,UINTA,663,356,79,116,161,$16,$133,$181
2603,WY,56043,WASHAKIE,341,173,33,54,86,$15,$138,$192


In [30]:
csr_clean = csr.copy()
csr_clean = csr_clean
for column in list(csr.columns)[3:]:
    csr_clean[column] = csr_clean[column].fillna('0')
    csr_clean[column] = csr_clean[column].apply(lambda x: float((str(x)).replace('$','').replace(',','')) )

In [31]:
merged_acs = all_acs.merge(csr_clean, how='left', left_on='County', right_on='FIPS County Code')
merged_acs

,DP03_0001E,DP03_0002E,DP03_0003E,DP03_0004E,DP03_0005E,DP03_0006E,DP03_0007E,DP03_0008E,DP03_0009E,DP03_0010E,...,FIPS County Code,County Name,Total Number of Consumers,Number of Consumers with CSR (AV of 73%/87%/94%),Number of Consumers with CSR AV of 73%,Number of Consumers with CSR AV of 87%,Number of Consumers with CSR AV of 94%,Average Monthly Advanced CSR Payment for Consumers with 73%,Average Monthly Advanced CSR Payment for Consumers with 87%,Average Monthly Advanced CSR Payment for Consumers with 94%
0,42712,26008,25699,24262,1437,309,16704,25699,0,22074,...,01001,AUTAUGA,1896.0,1341.0,142.0,399.0,800.0,13.0,87.0,117.0
1,160301,93872,93640,87753,5887,232,66429,93640,0,82813,...,01003,BALDWIN,11778.0,8342.0,1246.0,2426.0,4670.0,13.0,91.0,121.0
2,21476,10316,10316,8993,1323,0,11160,10316,0,9938,...,01005,BARBOUR,801.0,584.0,62.0,155.0,367.0,13.0,98.0,132.0
3,18496,9002,8997,8354,643,5,9494,8997,0,8514,...,01007,BIBB,629.0,477.0,58.0,121.0,298.0,15.0,101.0,123.0
4,46007,22969,22960,21593,1367,9,23038,22960,0,23370,...,01009,BLOUNT,2264.0,1663.0,197.0,437.0,1029.0,14.0,96.0,132.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,45100,17037,17037,14092,2945,0,28063,17037,0,23837,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3216,7229,3342,3342,2932,410,0,3887,3342,0,3622,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3217,19212,9278,9256,6960,2296,22,9934,9256,0,9970,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3218,28916,10802,10778,8171,2607,24,18114,10778,0,15074,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
missing_value_df = pd.DataFrame(merged_acs.isnull().sum())
missing_value_df #= missing_value_df.T
#all_merged = pd.concat([merged_acs, missing_value_df])

,0
DP03_0001E,0
DP03_0002E,0
DP03_0003E,0
DP03_0004E,0
DP03_0005E,0
...,...
Number of Consumers with CSR AV of 87%,622
Number of Consumers with CSR AV of 94%,622
Average Monthly Advanced CSR Payment for Consumers with 73%,622
Average Monthly Advanced CSR Payment for Consumers with 87%,622


In [33]:
merged_acs.fillna(0, inplace = True)
merged_acs.to_csv('county_characteristics.csv', index=False)